# Digit Recognizer

## Table of Contents
* [Introduction](#Introduction)
* [Data Preparation](#Data_Preparation)
    * [Importing the data](#Variance)
    * [Checking for missing values](#Missing)
    * [One hot encoding](#Onehot)
    * [Normalizing the data](#Codingpca)
* [Building the Models](#Building_the_Models)
    * [Simple model with data in array](#section_2_1)
    * [Simple model with data in matrix](#sub_section_2_1_1)
    * [Convolutional model](#sub_section_2_1_2)
* [Training the Models using Data Augmentation](#Training_the_Models_using_Data_Augmentation)
* [Evaluation of Models](#Evaluation_of_Models)
    * [Validation Curves](#section_3_1)
    * [Confusion Matrices](#sub_section_3_1_1)
* [Submissions](#Submissions)

## 1. Introduction <a class="anchor" id="Introduction"></a>

Here we present how different Neural Networks are predicting the digit shown in an image. We build, train and test 3 neural networks:
* Neural Network with image feeded as matrix: simple training set 0.97485, data augmentation 0.98317
* Neural Network with image feeded as an array: simple training set 0.97321, data augmentation 
* Convolutional Neural Network with image feeded as matrix: simple training set 0.99257, data augmentation 0.98317.

They all have 3 layers, and are trained with the same data using 20 epochs. It is interesting to note that the convolutional neural network did better without data augmentation, this could be because the model is overfitting eventhough we are changing the input images through data augmentation. It may just be because the model luckily turned out better with less data.  

Also, one final observation is that when testing the model on the validation set the convolutional neural network was the one that seemed to overfit the most the training data. However it was still better on the validation test than the other models.


## 2. Data Preparation <a class="anchor" id="Data_Preparation"></a>

### 2.1. Loading the data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dtrain = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
dtest = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
dtrain

In [ ]:
dtest

### 2.2. Checking for missing values

In [ ]:
for i in dtrain.isnull().any():
    if i == True:
        print(i) #Checking if there is any null values in the training set

There's no null values in the training set.

In [ ]:
for i in dtest.isnull().any():
    if i == True:
        print(i) #Checking if there is any null values in the test set

There's no null values in the test set.

Note from the dataset that for each image we have 784 pixels, this is because in the competition description we are told that the images are 28x28 pixel images. Hence lets transform the 784 length arrays into 28 by 28 matrixes. These describe better the images. First we must drop the label column in the training data.

In [ ]:
Y_train = dtrain["label"]
Y_train.value_counts()

There's a similar amount of images for each digit, which is good for training our model.


### 2.3. One hot encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder()
onehottrain = ohe.fit_transform(dtrain[['label']]).toarray()
onehottrain

In [ ]:
features_labels = ohe.categories_ 
print(features_labels)

In [ ]:
onehotfeatures = pd.DataFrame(np.array(onehottrain), columns= features_labels)
Y_train = onehotfeatures

In [ ]:
X_dtrain = dtrain.drop('label', axis = 1)

In [ ]:
X_dtrain.values

### 2.4. Normalization

We normalize the pixel values, since this makes the neural network perform better. 

In [ ]:
X_dtrain_normalized = X_dtrain / 255.0
dtest_normalized = dtest / 255.0

In [ ]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train_matrix = X_dtrain_normalized.values.reshape(-1,28,28,1)
dtest_matrix = dtest_normalized.values.reshape(-1,28,28,1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split the train and the validation set for the fitting
X_train0, X_val0, Y_train0, Y_val0 = train_test_split(X_train_matrix, Y_train.values, test_size = 0.1, random_state=2)
X_train1, X_val1, Y_train1, Y_val1 = train_test_split(X_dtrain_normalized.values, Y_train.values, test_size = 0.1, random_state=2)

In [ ]:
plt.imshow(X_train0[2][:,:,0])

In [ ]:
X_train1.shape

## 3. Building & Training the Models <a class="anchor" id="Building_the_Models"></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Reshape, Flatten
from tensorflow.keras.activations import linear, relu, sigmoid

### 3.1. Simple model with data in array

In [ ]:
tf.random.set_seed(1234) # for consistent results
simplemodel = Sequential(
    [        
        tf.keras.Input(shape=(784,)),     # This specifies the shape our training set elements
        tf.keras.layers.Dense(256, activation='relu', name = "L1"), #  
        tf.keras.layers.Dense(15, activation='relu',  name = "L2"), #   
        tf.keras.layers.Dense(10, activation='softmax', name = "L3"),  #  
    ], name = "model" 
)
simplemodel.summary()

In [ ]:
simplemodel.compile(
    loss='categorical_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
     metrics=["accuracy"]
)
# from_logits=True creates a softmax function and .Adam chooses the Adaptive Moment optimizer.
# I use 'categorical_crossentropy' instead of sparse categorical_crossentropy because we have onehotencoded the data
# metrics=['accuracy'] makes the model calculate its own accuracy automatically (Stored in .history)
simplemodeldataaug = simplemodel

In [ ]:
Y_train1.shape

In [ ]:
simple = simplemodel.fit(X_train1, Y_train1, validation_data = (X_val1,Y_val1), epochs = 20)

### 3.2. Simple model with data in matrix

In [ ]:
tf.random.set_seed(1234) # for consistent results
matrixmodel = Sequential(
    [        
        tf.keras.Input(shape=(28,28)),     # This specifies the shape our training set elements
        Flatten(),
        tf.keras.layers.Dense(256, activation='relu', name = "L1"), #  
        tf.keras.layers.Dense(15, activation='relu',  name = "L2"), #   
        tf.keras.layers.Dense(10, activation='softmax', name = "L3"),  #  
    ], name = "model" 
)
matrixmodel.summary()

In [ ]:
matrixmodel.compile(
    loss='categorical_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
     metrics=["accuracy"]
)
# from_logits=True creates a softmax function and .Adam chooses the Adaptive Moment optimizer.
# I use 'categorical_crossentropy' instead of sparse categorical_crossentropy because we have onehotencoded the data
matrixmodeldataaug = matrixmodel

In [ ]:
matrix = matrixmodel.fit(X_train0, Y_train0, validation_data = (X_val0,Y_val0), epochs = 20)

### 3.3. Convolutional model with data in matrix

In [ ]:

convmodel = Sequential([
  tf.keras.Input(shape=(28,28,1)),
  tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5), padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
  Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax', name = "L3")
])
convmodel.summary()

In [ ]:
convmodel.compile(
    loss='categorical_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
     metrics=["accuracy"]
)
convmodeldataaug = convmodel

In [ ]:
print(X_train0[0].shape)

In [ ]:
print(Y_train0[0].shape)

In [ ]:
conv = convmodel.fit(X_train0, Y_train0, validation_data = (X_val0,Y_val0), epochs = 20)

## 4. Training the Models using Data Augmentation <a class="anchor" id="Training_the_Models_using_Data_Augmentation"></a>

Data augmentation is a method to increase the traing set size, it makes the model be able to adapt better for different types of inputs. We can do data augmentation here since one can edit images (rotate, filter,...) in training set to get new images and include them in the training set. 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [ ]:
# With data augmentation to prevent overfitting (accuracy 0.99286)

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen1 = datagen
datagen.fit(X_train0)

In [ ]:
# Fit the models
matrixdataaug_hist = matrixmodeldataaug.fit_generator(datagen.flow(X_train0,Y_train0, batch_size=90),
                               validation_data = (X_val0,Y_val0), epochs =20,  steps_per_epoch=X_train0.shape[0] // 90
                              )
convdataaug_hist = convmodeldataaug.fit_generator(datagen.flow(X_train0,Y_train0, batch_size=90),
                             validation_data = (X_val0,Y_val0),
                              epochs = 20, steps_per_epoch=X_train0.shape[0] // 90
                              )

## 5. Evaluation of Models <a class="anchor" id="Evaluation_of_Models"></a>

### 5.1. Validation curves

In [ ]:
history_dict1 = simple.history
print(history_dict1.keys())

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(simple.history['loss'], color='b', label="Training loss")
ax[0].plot(simple.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(simple.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(simple.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
history_dict0 = matrix.history
print(history_dict0.keys())

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(matrix.history['loss'], color='b', label="Training loss")
ax[0].plot(matrix.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(matrix.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(matrix.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
history_dictconv = conv.history
print(history_dictconv.keys())

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(conv.history['loss'], color='b', label="Training loss")
ax[0].plot(conv.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(conv.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(conv.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

From the graphs we can observe there is overfitting. We could perform some chages in the training method to prevent this.

### 5.2. Confusion Matrices

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = simplemodel.predict(X_val1)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val1,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = matrixmodeldataaug.predict(X_val0)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val0,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix convolutional model',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = convmodeldataaug.predict(X_val0)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val0,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

In [ ]:
# Display some error results 

# Errors are difference between predicted labels and true labels
errors = (Y_pred_classes - Y_true != 0)

Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_val_errors = X_val1[errors]

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    """ This function shows 6 images with their predicted and real labels"""
    n = 0
    nrows = 2
    ncols = 3
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True)
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]).reshape((28,28)))
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(pred_errors[error],obs_errors[error]))
            n += 1

# Probabilities of the wrong predicted numbers
Y_pred_errors_prob = np.max(Y_pred_errors,axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 6 errors 
most_important_errors = sorted_dela_errors[-6:]

# Show the top 6 errors
display_errors(most_important_errors, X_val_errors, Y_pred_classes_errors, Y_true_errors)

## 6. Submissions <a class="anchor" id="Submissions"></a>

In [ ]:
# predict results
results1 = simplemodel.predict(dtest_normalized)

# select the indix with the maximum probability
predictions1 = np.argmax(results1,axis = 1)

predictions1 = pd.Series(predictions1,name="Label")

In [ ]:
# predict results
results0 = matrixmodel.predict(dtest_matrix)

# select the indix with the maximum probability
predictions0 = np.argmax(results0,axis = 1)

predictions0 = pd.Series(predictions0,name="Label")

In [ ]:
# predict results
resultsconv = convmodel.predict(dtest_matrix)

# select the indix with the maximum probability
predictionsconv = np.argmax(resultsconv,axis = 1)

predictionsconv = pd.Series(predictionsconv,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),predictions1],axis = 1)
submission.to_csv("submission.csv",index=False)

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),predictions0],axis = 1)
submission.to_csv("submission0.csv",index=False)

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),predictionsconv],axis = 1)
submission.to_csv("submissionconv.csv",index=False)

In [ ]:
# predict results
results = matrixmodeldataaug.predict(dtest_matrix)

# select the indix with the maximum probability
predictions = np.argmax(results,axis = 1)

predictions = pd.Series(predictions,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),predictions],axis = 1)
submission.to_csv("submissionmatrixdataaug.csv",index=False)

In [ ]:
# predict results
results = convmodeldataaug.predict(dtest_matrix)

# select the indix with the maximum probability
predictions = np.argmax(results0,axis = 1)

predictions = pd.Series(predictions,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),predictions],axis = 1)
submission.to_csv("submissionconvdataaug.csv",index=False)